In [13]:
import json

from kafka import KafkaConsumer

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [14]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Gabriel',
    last_name='Avinaz'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Gabriel',
 'last_name': 'Avinaz',
 'client_id': 'AvinazGabriel',
 'topic_prefix': 'AvinazGabriel'}

Create a consumer without subscribing to any particular topic

In [15]:
general_consumer = KafkaConsumer(
    bootstrap_servers=config['bootstrap_servers']
)

List all topics you are currently allowed to view

In [16]:
general_consumer.topics()

{'AvinazGabriel-accelerations',
 'AvinazGabriel-locations',
 'ClaytonDaniel-accelerations',
 'ClaytonDaniel-locations',
 'ShermanMilan-accelerations',
 'ShermanMilan-locations',
 'TabbalatAbed-accelerations',
 'TabbalatAbed-locations',
 'TabbalatAbed-simple',
 'WeatherfordPatrick-accelerations',
 'WeatherfordPatrick-locations'}

Close the consumer, waiting indefinitely for any needed cleanup.

In [17]:
general_consumer.close()

In [18]:
def create_kafka_consumer(topics, config=config):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']    
    topic_list = ['{}-{}'.format(topic_prefix, topic) for topic in topics]
    
    return KafkaConsumer(
        *topic_list,
        client_id=client_id,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        enable_auto_commit=False,
        value_deserializer=lambda x: json.loads(x)
    )

consumer = create_kafka_consumer(['locations', 'accelerations'])

Gets a list of this consumer's current subscriptions

In [19]:
consumer.subscription()

{'AvinazGabriel-accelerations', 'AvinazGabriel-locations'}

The following function prints messages from the current consumer subscriptions. It will continue until manually stopped. 

In [ ]:
def print_messages(consumer=consumer):
    try:
        for message in consumer:
                msg_metadata = 'Message metadata: {}:{}:{}'.format(
                    message.topic, message.partition, message.offset
                )

                if message.key is not None:
                    msg_key = message.key.decode('utf-8')
                else:
                    msg_key = ''
                msg_value = json.dumps(message.value, indent=2)
                msg_value = '\n'.join(['  {}'.format(value) for value in msg_value.split('\n')])

                print('Message metadata:')
                print('  Topic: {}'.format(message.topic))
                print('  Partition: {}'.format(message.partition))
                print('  Offset: {}'.format(message.offset))
                print('Message Key: {}'.format(msg_key))
                print('Message Value:')
                print(msg_value)
                print()
    except KeyboardInterrupt:
        print("STOPPING MESSAGE CONSUMER")
        
print_messages()

Message metadata:
  Topic: AvinazGabriel-locations
  Partition: 0
  Offset: 0
Message Key: 501f103d7daa47b78c4ff397db4537b4
Message Value:
  {
    "key1": "value1",
    "key2": "value2"
  }

Message metadata:
  Topic: AvinazGabriel-locations
  Partition: 0
  Offset: 1
Message Key: 8a8a75040bed459cb890ffb21260fa05
Message Value:
  {
    "id": "d602b69361b975ad36de1c0c347b11c7",
    "ride_id": "6ef732ea5d79eeb952e859d1accf0f04",
    "uuid": "7901ba7a61704c1bb310802ea69b8a86",
    "timestamp": "1970-01-01T00:25:07.034Z",
    "offset": 102.57438001660061,
    "course": 174.02879333496094,
    "latitude": 40.75754753567096,
    "longitude": -73.79306668914728,
    "geohash": "dr5xbg9tbvt3",
    "speed": 7.010000228881836,
    "accuracy": 10.0,
    "timelapse": false,
    "filename": "2330d18d-4f74-45df-b7bf-1178e19ee7ad.mov",
    "t": 102.5
  }

Message metadata:
  Topic: AvinazGabriel-locations
  Partition: 0
  Offset: 2
Message Key: 11d292fa3aba4982be9d999ab15442a8
Message Value:
  {
    

Close the consumer, waiting indefinitely for any needed cleanup.

In [10]:
consumer.close()